In [1]:
from kaggle_environments import evaluate, make, utils
import random
import numpy as np

env = make("connectx", debug=True)

/Users/jordanbonil/Documents/source/git/connextx/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


termcolor not installed, skipping dependency
No pygame installed, ignoring import


In [2]:
help(env)

Help on Environment in module kaggle_environments.core object:

class Environment(builtins.object)
 |  Environment(specification=None, configuration=None, info=None, steps=None, logs=None, agents=None, interpreter=None, renderer=None, html_renderer=None, debug=False, state=None)
 |
 |  Methods defined here:
 |
 |  __init__(self, specification=None, configuration=None, info=None, steps=None, logs=None, agents=None, interpreter=None, renderer=None, html_renderer=None, debug=False, state=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  clone(self)
 |      Returns:
 |          Environment: A copy of the current environment.
 |
 |  debug_print(self, message)
 |
 |  play(self, agents=None, **kwargs)
 |      Renders a visual representation of the environment and allows interactive action selection.
 |
 |      Args:
 |          **kwargs (dict): Args directly passed into render().  Mode is fixed to ipython.
 |
 |      Returns:
 |          None: prints directly

In [ ]:
env.state[1]['observation']

In [8]:
env.step(4)
env.render(mode="ipython", width=500, height=450)

TypeError: object of type 'int' has no len()

In [ ]:
# This agent random chooses a non-empty column.
def my_agent(observation, configuration):
    from random import choice
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

# Play as first position against random agent.
trainer = env.train([None, "random"])

observation = trainer.reset()

while not env.done:
    my_action = my_agent(observation, env.configuration)
    print("My Action", my_action)
    observation, reward, done, info = trainer.step(my_action)
    # env.render(mode="ipython", width=100, height=90, header=False, controls=False)
env.render(mode="ipython", width=500, height=450)

In [ ]:
env.play([None, "negamax"], width=500, height=450)

In [ ]:
env.step()

In [ ]:
dir(env)

In [3]:
# build your model

# It will be a Q learning model

# it will have parameters:
# - alpha (learning rate)
# - Q(s,.) initialised for every state, so choose the quality of the state. except for terminal states, where Q(terminal = 0)
# - some gamma (importance of future rewards) (discount factor)
# - a reward function

def preprocess_state(board, mark):
    board_array = np.array(board).flatten()
    state = np.concatenate([board_array, [mark]])
    return state


def is_terminal():
     #env.is_terminal or something
     pass


def get_next_state(state, action, piece, config):
    next_state = state.copy()
    for row in range(config.rows-1, -1, -1):
        if next_state[row][action] == 0:
            break
    next_state[row][action] = piece
    return next_state


def get_reward(state):
    if is_terminal(state):
        # if won
            return 1
        # else we lost:
            return -1
    return 0

class QTable:
    def __init__(self, n_action_space):
        self.table = dict()
        self.action_space = n_action_space
        
    def add_item(self, state_key):
        self.table[state_key] = list(np.zeros(self.action_space))
        
    def __call__(self, state):
        board = state.board[:] # Get a copy
        board.append(state.mark)
        state_key = np.array(board).astype(str)
        state_key = hex(int(''.join(state_key), 3))[2:]
        if state_key not in self.table.keys():
            self.add_item(state_key)
        
        return self.table[state_key]
        

class QLearner():
    def __init__(self, q_table, alpha= 0.1, gamma=0.5, epsilon = 0.1):
        self.alpha = alpha # float  
        self.gamma = gamma # float
        self.q = q_table # func or dict table (initial quality table of every state but state space is biiiiig) dict of dict(float)
        self.epsilon  = epsilon # float, policy parameter eps-greedy exploration

    def choose_action(self, state, valid_moves, learning = False): # eps-greedy policy        
        if learning and random.random() < self.epsilon:
            action = random.choice(valid_moves) # exploration
        else:
            move_quality = sorted([(self.q(state, act), act) for act in valid_moves]) # ascending sort
            action = move_quality[-1][1] # the best quality is last, the action is the column id on the idx = 1 element of move_quality
        return action
    
    def update_quality(self, action, state):
        # action is a move
        # state is a tbale of connectx
        if is_terminal(state):
            self.q(state)[:] = 0
        
        else:
            new_state = get_next_state(state, action)
            self.q(state)[action] += + self.alph * (get_reward(new_state) + self.gamma * max(self.q(new_state)) - self.q(state)[action])




In [ ]:
alpha = 0.1
gamma = 0.6
epsilon = 0.99
min_epsilon = 0.1

episodes = 10000

alpha_decay_step = 1000
alpha_decay_rate = 0.9
epsilon_decay_rate = 0.9999

In [ ]:
# all encompassing for loop
q_table = QTable(env.configuration['columns'])
q_learner = QLearner()

games = 1000
plays = 1000

for g in range(games):

    state = env.reset()
    epochs, total_reward = 0, 0
    done = False

    for p in range(plays):


        
        # read state

        # pick a move

        # make a move

        # observe the next state

        # update the DynamicQTable
        



In [ ]:
def q_agent(obs, config):
    # intiialise the learner
    q_player = QLearner(q_init)

    # Prepare state and valid moves
    state = np.array(obs.board)
    valid_moves = [col for col in range(config.columns) if state[col] == 0]
    # state = preprocess_state(obs.board, obs.mark)

    return q_player.choose_action(state, valid_moves, learning = False)
    

In [4]:
# make a function to train your model

def train_model():
    # read the state (the board)

    # decide on an action

    # read the consequence of the action

    # adapt the model



In [5]:
# save your model's weights/parameters in a local file (json)

In [6]:
# wrtie a submission.py file with the model declaration
# set the model parameters/weights using the saved json post training
# define the agent's actions according to the model
# et voile


In [ ]:
def write_agent_to_file(filename="submission.py", **params):
    with open(filename, "w") as f:
        f.write("""
#### MAKE FULL MODEL ####
                
#### IMPORT MODEL PARAMETERS FROM LOCAL JSON FILE####

def my_agent(obs, config):
    # Initialize and load the Actor model
    actor = Actor(input_shape=43, output_shape=config.columns)
    load_weights_into_model(actor, actor_weights)
    actor.eval()

    critic = Critic(input_shape=43)
    load_weights_into_model(critic, critic_weights)
    critic.eval()
    
    # Prepare state and valid moves
    board = np.array(obs.board)
    valid_moves = [col for col in range(config.columns) if board[col] == 0]
    state = preprocess_state(obs.board, obs.mark)
    
    
    action = get_action(state, valid_moves, board.flatten(), obs.mark, 3-obs.mark, config.rows, config.columns)
    # Mask invalid moves
    if action not in valid_moves:
        action = random.choice(valid_moves)
    return int(action)
        """)
    print(f"Submission file {filename} created successfully!")


write_agent_to_file(filename="submission.py")

In [ ]:
# Observe your model play against itself

from kaggle_environments import make
import sys

# Import the agent directly
from submission import my_agent

# Run the environment with the agent
env = make("connectx", debug=True)
env.run([my_agent, my_agent])

print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

# Display the game
env.render(mode="ipython")

In [ ]:
random.random()
